# 🦴 脊椎椎體頂點檢測 - Colab 推理伺服器
## Spine Vertebra Corner Detection - Inference Server

**使用方式：**
1. 依序執行每個 Cell
2. 上傳模型檔 `best_vertebra_model.pth` 和程式碼
3. 啟動伺服器後，點擊 ngrok 公開網址即可使用

## Step 1: 安裝依賴套件

In [ ]:
!pip install -q fastapi uvicorn[standard] python-multipart pydicom albumentations pyngrok

## Step 2: 上傳檔案

執行下方 Cell，會跳出檔案選擇器。請上傳以下檔案：
- `api_server_vertebra.py`
- `inference_vertebra.py`
- `train_vertebra_model.py`
- `spine-inference-web.html`
- `spinal-annotation-web.html`
- `best_vertebra_model.pth` (~239 MB，需要等待)

In [ ]:
from google.colab import files
import os

# 切換到工作目錄
os.makedirs('/content/spine', exist_ok=True)
os.chdir('/content/spine')

print('請上傳以下檔案：')
print('  1. api_server_vertebra.py')
print('  2. inference_vertebra.py')
print('  3. train_vertebra_model.py')
print('  4. spine-inference-web.html')
print('  5. spinal-annotation-web.html')
print('  6. best_vertebra_model.pth')
print()

uploaded = files.upload()

print(f'\n✅ 已上傳 {len(uploaded)} 個檔案：')
for name, data in uploaded.items():
    size_mb = len(data) / 1024 / 1024
    print(f'   {name} ({size_mb:.1f} MB)')

### (替代方案) 從 Google Drive 載入
如果檔案已在 Google Drive，可以用以下方式掛載：

In [ ]:
# === 替代方案：從 Google Drive 載入 ===
# 取消註解以下程式碼，並修改路徑

# from google.colab import drive
# drive.mount('/content/drive')

# # 複製檔案到工作目錄
# import shutil
# DRIVE_DIR = '/content/drive/MyDrive/Spine'  # <-- 修改為你的 Drive 路徑
# os.makedirs('/content/spine', exist_ok=True)
# for f in os.listdir(DRIVE_DIR):
#     src = os.path.join(DRIVE_DIR, f)
#     dst = os.path.join('/content/spine', f)
#     if os.path.isfile(src):
#         shutil.copy2(src, dst)
#         print(f'  複製 {f}')
# os.chdir('/content/spine')

## Step 3: 驗證檔案

In [ ]:
import os
os.chdir('/content/spine')

required_files = [
    'api_server_vertebra.py',
    'inference_vertebra.py',
    'train_vertebra_model.py',
    'spine-inference-web.html',
    'spinal-annotation-web.html',
    'best_vertebra_model.pth',
]

all_ok = True
for f in required_files:
    exists = os.path.exists(f)
    size = os.path.getsize(f) / 1024 / 1024 if exists else 0
    status = f'✅ {size:.1f} MB' if exists else '❌ 缺少'
    print(f'  {f}: {status}')
    if not exists:
        all_ok = False

if all_ok:
    print('\n🎉 所有檔案就緒！')
else:
    print('\n⚠️ 有檔案缺少，請回到 Step 2 上傳')

## Step 4: 測試模型載入

In [ ]:
import sys
sys.path.insert(0, '/content/spine')
os.chdir('/content/spine')

import torch
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

from inference_vertebra import VertebraInference
analyzer = VertebraInference('best_vertebra_model.pth', device='auto')
print(f'\n✅ 模型載入成功！Device: {analyzer.device}')

## Step 5: 啟動推理伺服器

使用 **ngrok** 建立公開網址，讓你可以從任何裝置存取。

> ⚠️ 第一次使用需要免費註冊 ngrok：https://dashboard.ngrok.com/signup
> 取得 authtoken 後填入下方

In [ ]:
# === 設定 ngrok authtoken ===
# 前往 https://dashboard.ngrok.com/get-started/your-authtoken 取得
NGROK_TOKEN = ''  # <-- 填入你的 ngrok authtoken

import subprocess
import threading
import time

os.chdir('/content/spine')

# 啟動 FastAPI server (背景執行)
server_process = subprocess.Popen(
    [sys.executable, 'api_server_vertebra.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)  # 等待伺服器啟動

# 檢查伺服器是否啟動
import urllib.request
try:
    resp = urllib.request.urlopen('http://localhost:8001/health')
    print(f'✅ 伺服器啟動成功: {resp.read().decode()}')
except Exception as e:
    print(f'❌ 伺服器啟動失敗: {e}')
    print('stderr:', server_process.stderr.read().decode()[-500:])
    raise

# 建立 ngrok tunnel
if NGROK_TOKEN:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(8001)
    print(f'\n{"="*50}')
    print(f'  🌐 公開網址: {public_url}')
    print(f'  📱 可從手機或其他電腦存取')
    print(f'{"="*50}')
else:
    print('\n⚠️ 未設定 NGROK_TOKEN')
    print('  本地存取: http://localhost:8001')
    print('  要從外部存取，請設定 ngrok token')
    print('  前往 https://dashboard.ngrok.com/signup 免費註冊')

## (替代) 不用 ngrok，使用 Colab 內建 tunnel

In [ ]:
# === 替代方案：使用 localtunnel (不需註冊) ===
# 取消註解以下程式碼

# !npm install -g localtunnel > /dev/null 2>&1

# import subprocess
# lt_process = subprocess.Popen(
#     ['lt', '--port', '8001'],
#     stdout=subprocess.PIPE,
#     stderr=subprocess.PIPE
# )
# time.sleep(3)
# output = lt_process.stdout.readline().decode()
# print(f'🌐 Localtunnel URL: {output}')

## Step 6: 停止伺服器
完成後執行此 Cell 停止伺服器

In [ ]:
try:
    server_process.terminate()
    print('✅ 伺服器已停止')
except:
    print('伺服器未在執行')

try:
    from pyngrok import ngrok
    ngrok.kill()
    print('✅ ngrok tunnel 已關閉')
except:
    pass